# TMDB API로 데이터 수집

## 환경 설정

In [2]:
# 라이브러리 불러오기
import os
import time
import random
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor
import requests
from requests.adapters import HTTPAdapter, Retry
import pandas as pd
from dotenv import load_dotenv

# 환경 설정 
load_dotenv()
API_KEY = os.getenv("TMDB_API_KEY")

if not API_KEY:
    raise ValueError("API 키가 없습니다! .env 파일 확인하세요.")

BASE_URL = "https://api.themoviedb.org/3"

# 세션 + 재시도 설정 (네트워크 에러 대비)
session = requests.Session()
retries = Retry(
    total=5,
    backoff_factor=1,
    status_forcelist=[429, 500, 502, 503, 504],
)
adapter = HTTPAdapter(max_retries=retries)
session.mount("https://", adapter)
session.mount("http://", adapter)


# 함수

In [7]:
# ==================== 공통 함수 ====================
def tmdb_get(path, params=None):
    """
    TMDB GET 요청 공통 함수
    - 자동으로 api_key 포함
    - 에러 시 None 리턴
    """
    if params is None:
        params = {}
    params["api_key"] = API_KEY
    params.setdefault("language", "en-US")
    
    url = f"{BASE_URL}{path}"
    
    try:
        resp = session.get(url, params=params, timeout=10)
        resp.raise_for_status()
        return resp.json()
    except Exception as e:
        print(f"[tmdb_get error] {url} / {e}")
        return None


# ==================== 1. 영화 리스트 수집 ====================
def fetch_movies_by_year(year, top_n=500, start_date=None, end_date=None):
    """
    특정 연도(year)에 대해 popularity/vote_count 상위 영화 리스트 조회.
    - TMDB discover/movie 사용
    - 최대 top_n개까지 가져옴
    """
    results = []
    page = 1
    per_page = 20  # TMDB 한 페이지당 20편
    
    while len(results) < top_n:
        params = {
            "sort_by": "vote_count.desc",  # 인기순
            "page": page,
            "include_adult": True,  # 성인 영화 포함 (불리언으로 수정)
            "include_video": False,
        }
        
        # 기본은 연도만, 필요시 날짜 범위 지정
        if start_date or end_date:
            if start_date:
                params["primary_release_date.gte"] = start_date
            if end_date:
                params["primary_release_date.lte"] = end_date
        else:
            params["primary_release_year"] = year
        
        data = tmdb_get("/discover/movie", params=params)
        if not data or "results" not in data:
            break
        
        page_results = data["results"]
        if not page_results:
            break
        
        for m in page_results:
            results.append({
                "id": m.get("id"),
                "title": m.get("title"),
                "original_title": m.get("original_title"),
                "original_language": m.get("original_language"),
                "release_date": m.get("release_date"),
                "popularity": m.get("popularity"),
                "vote_count": m.get("vote_count"),
                "vote_average": m.get("vote_average"),
                "genre_ids": m.get("genre_ids", []),
                "adult": m.get("adult"),
                "video": m.get("video"),
            })
        
        if len(results) >= top_n:
            break
        
        total_pages = data.get("total_pages", 1)
        if page >= total_pages:
            break
        
        page += 1
        time.sleep(0.05)  # discover 속도 조절
    
    return results[:top_n]


# ==================== 2. 영화 상세 정보 수집 ====================
def fetch_movie_full(movie_id: int):
    """
    한 편의 영화에 대해 상세 정보 수집.
    /movie/{id}?append_to_response=credits,keywords,release_dates,watch/providers
    문제가 생기면 None 리턴
    """
    params = {
        "append_to_response": "credits,keywords,release_dates,watch/providers"
    }
    
    data = tmdb_get(f"/movie/{movie_id}", params=params)
    if not data:
        return None
    
    # ===== 장르 / 제작사 / 나라 / 언어 등 =====
    genres = [g.get("name") for g in data.get("genres", [])]
    prod_companies = [c.get("name") for c in data.get("production_companies", [])]
    prod_countries = [c.get("name") for c in data.get("production_countries", [])]
    spoken_languages = [l.get("name") for l in data.get("spoken_languages", [])]
    
    # ===== 키워드 =====
    keywords_block = data.get("keywords", {})
    if isinstance(keywords_block, dict):
        keywords_list = [k.get("name") for k in keywords_block.get("keywords", [])]
    else:
        keywords_list = []
    
    # ===== Credits (cast, crew) =====
    credits = data.get("credits", {})
    cast_list = credits.get("cast", []) if isinstance(credits, dict) else []
    crew_list = credits.get("crew", []) if isinstance(credits, dict) else []
    
    # 상위 배우 3명
    top_cast = [c.get("name") for c in cast_list[:3]]
    
    # 감독
    directors = [c.get("name") for c in crew_list if c.get("job") == "Director"]
    
    # 작가 (Writer, Screenplay)
    writers = [
        c.get("name") for c in crew_list
        if c.get("department") == "Writing" or c.get("job") in ("Writer", "Screenplay")
    ]
    
    # ===== OTT 제공 정보 (watch/providers) =====
    providers_data = data.get("watch/providers", {})
    providers_results = providers_data.get("results", {}) if isinstance(providers_data, dict) else {}
    
    # 국가별 OTT 정보 (flatrate = 구독형)
    global_providers = {}
    for country, info in providers_results.items():
        flatrate = info.get("flatrate", [])
        provider_names = [p["provider_name"] for p in flatrate]
        if provider_names:
            global_providers[country] = provider_names
    
    # ===== 최종 결과 딕셔너리 =====
    result = {
        "id": data.get("id"),
        "imdb_id": data.get("imdb_id"),  # IMDb ID 유지
        "title": data.get("title"),
        "original_title": data.get("original_title"),
        "original_language": data.get("original_language"),
        "spoken_languages": spoken_languages,
        "overview": data.get("overview"),
        "tagline": data.get("tagline"),
        "release_date": data.get("release_date"),
        "runtime": data.get("runtime"),
        "budget": data.get("budget"),
        "revenue": data.get("revenue"),
        "popularity": data.get("popularity"),
        "vote_count": data.get("vote_count"),
        "vote_average": data.get("vote_average"),
        "adult": data.get("adult"),
        "genres": genres,
        "genre_ids": [g.get("id") for g in data.get("genres", [])],  # 머신러닝용
        "production_companies": prod_companies,
        "production_countries": prod_countries,
        "keywords": keywords_list,
        "top_cast": top_cast,
        "directors": directors,
        "writers": writers,
        "providers": global_providers,
    }
    
    return result


# ==================== 3. 멀티스레드 상세 수집 ====================
def fetch_one(mid):
    """
    멀티스레드에서 쓸 래퍼 함수
    - 에러 나면 None 리턴
    """
    try:
        return fetch_movie_full(mid)
    except Exception as e:
        print(f"[error] id={mid}, {e}")
        return None


def collect_details_parallel(movie_ids, max_workers=5):
    """
    movie_ids 리스트를 받아서 병렬로 상세 정보 수집
    """
    detail_rows = []
    total = len(movie_ids)
    start = time.time()
    
    print(f"[detail] 전체 대상 편수: {total}, 워커 수: {max_workers}")
    
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        # map을 사용하면 순서 보장 (as_completed 대신)
        results = list(executor.map(fetch_one, movie_ids))
        
        for i, result in enumerate(results, start=1):
            if result is not None:
                detail_rows.append(result)
            
            if i % 50 == 0 or i == total:
                elapsed = time.time() - start
                print(f"[detail] {i} / {total} 편 처리 완료 (경과: {elapsed/60:.1f}분)")
    
    return detail_rows


# ==================== 4. 연도별 통합 수집 함수 ====================
def collect_movies_full(start_year=2005, end_year=2024, top_n_per_year=500):
    """
    start_year ~ end_year까지 연도별로 top_n_per_year씩 수집
    - 1단계: fetch_movies_by_year (기본 리스트)
    - 2단계: fetch_movie_full (상세 정보) - 병렬 처리
    - 중복 제거 후 최종 DataFrame 반환
    """
    all_list_rows = []
    
    # ===== 1단계: 연도별 리스트 수집 =====
    for year in range(start_year, end_year + 1):
        print(f"\n{'='*60}")
        print(f"[list] {year}년 수집 시작...")
        print(f"{'='*60}")
        
        movies = fetch_movies_by_year(year, top_n=top_n_per_year)
        
        for m in movies:
            m["release_year"] = year
        
        all_list_rows.extend(movies)
        print(f"[list] {year}년 수집 완료: {len(movies)}편")
    
    df_list = pd.DataFrame(all_list_rows)
    print(f"\n연도별 Top 리스트(중복 포함) 행 수: {len(df_list)}")
    
    # ===== ID 기준 중복 제거 =====
    before = len(df_list)
    df_list = df_list.drop_duplicates(subset=["id"])
    after = len(df_list)
    print(f"ID 기준 중복 제거: {before} → {after}")
    
    # ===== 2단계: 상세 정보 수집 (병렬) =====
    movie_ids = df_list["id"].astype(int).tolist()
    detail_rows = collect_details_parallel(movie_ids, max_workers=5)
    
    df_detail = pd.DataFrame(detail_rows)
    print(f"\n상세 정보 수집 행 수: {len(df_detail)}")
    
    # ===== 3단계: df_list와 df_detail 병합 =====
    # df_list에는 release_year, 기본 리스트 정보가 있고
    # df_detail에는 상세 정보만 있음
    # 중복 컬럼 제거 후 left merge
    df_final = pd.merge(
        df_detail,
        df_list[["id", "release_year"]],  # 중복 컬럼 제거
        on="id",
        how="left"
    )
    
    print(f"\n최종 데이터프레임 행/열: {df_final.shape}")
    return df_final


# ==================== 5. 코로나 종료 이후 무작위 샘플 수집 ====================
def collect_post_covid_sample(sample_size=500):
    """
    2023년 5월 5일 (WHO 팬데믹 종료 선언) 이후 ~ 현재까지 무작위 샘플 수집
    - 멀티 스레드로 랜덤 페이지 병렬 수집
    - 중복 방지를 위해 set 사용
    """
    print(f"\n{'='*60}")
    print(f"[post-covid] 2023-05-05 (WHO 종료 선언) 이후 무작위 {sample_size}건 샘플 수집 시작")
    print(f"{'='*60}")
    
    # 전체 페이지 수 확인
    params = {
        "sort_by": "popularity.desc",
        "primary_release_date.gte": "2023-05-05",
        "primary_release_date.lte": datetime.now().strftime("%Y-%m-%d"),
        "include_adult": True,
        "page": 1
    }
    data = tmdb_get("/discover/movie", params=params)
    total_pages = data.get("total_pages", 1) if data else 1
    
    # TMDB API 제한: 페이지 500까지만 접근 가능
    max_page = min(total_pages, 500)
    
    print(f"전체 페이지 수: {total_pages} (접근 가능: {max_page})")
    
    # ===== 멀티 스레드로 랜덤 페이지 수집 =====
    def fetch_random_page(page_num):
        """단일 페이지 수집"""
        params_copy = params.copy()
        params_copy["page"] = page_num
        data = tmdb_get("/discover/movie", params=params_copy)
        if data and "results" in data:
            return [m.get("id") for m in data["results"]]
        return []
    
    # 필요한 페이지 수 계산 (1페이지당 20개, 여유있게 수집)
    pages_needed = (sample_size // 20) + 10  # 중복 고려해서 여유있게
    random_pages = random.sample(range(1, max_page + 1), min(pages_needed, max_page))
    
    print(f"랜덤 페이지 {len(random_pages)}개에서 영화 ID 수집 중...")
    
    movie_ids_set = set()
    
    with ThreadPoolExecutor(max_workers=10) as executor:
        results = executor.map(fetch_random_page, random_pages)
        
        for page_ids in results:
            movie_ids_set.update(page_ids)
            if len(movie_ids_set) >= sample_size:
                break
    
    movie_ids = list(movie_ids_set)[:sample_size]
    print(f"수집된 고유 영화 ID 수: {len(movie_ids)}")
    
    # 상세 정보 수집 (기존 멀티 스레드 방식)
    detail_rows = collect_details_parallel(movie_ids, max_workers=5)
    df_sample = pd.DataFrame(detail_rows)
    
    # release_year 추가
    df_sample["release_year"] = pd.to_datetime(
        df_sample["release_date"], errors="coerce"
    ).dt.year
    
    print(f"최종 샘플 데이터 행/열: {df_sample.shape}")
    return df_sample


## 실행

In [8]:
# ==================== 6. 실행 ====================
if __name__ == "__main__":
    print("="*60)
    print("🎬 TMDB 영화 데이터 수집 시작")
    print("📅 수집 기간: 2023년 5월 5일 (WHO 팬데믹 종료 선언) 이후")
    print("📊 수집 목표: 무작위 500편")
    print("="*60)
    
    # ===== 2023년 5월 5일 이후 무작위 샘플 500편 수집 =====
    df_final = collect_post_covid_sample(sample_size=500)
    
    # ===== CSV 저장 =====
    output_path = "tmdb_movies_post_covid_may2023_500.csv"
    df_final.to_csv(output_path, index=False, encoding="utf-8-sig")
    
    print("\n" + "="*60)
    print("🎉 수집 완료!")
    print(f"📊 최종 데이터: {len(df_final)}편")
    print(f"💾 저장 완료: {output_path}")
    print("="*60)
    
    # ===== 간단한 통계 출력 =====
    print("\n[데이터 요약]")
    print(df_final[["id", "title", "release_year", "revenue", "budget", "popularity"]].head(10))
    print(f"\n컬럼 목록: {list(df_final.columns)}")

🎬 TMDB 영화 데이터 수집 시작
📅 수집 기간: 2023년 5월 5일 (WHO 팬데믹 종료 선언) 이후
📊 수집 목표: 무작위 500편

[post-covid] 2023-05-05 (WHO 종료 선언) 이후 무작위 500건 샘플 수집 시작
전체 페이지 수: 6377 (접근 가능: 500)
랜덤 페이지 35개에서 영화 ID 수집 중...
수집된 고유 영화 ID 수: 500
[detail] 전체 대상 편수: 500, 워커 수: 5
[detail] 50 / 500 편 처리 완료 (경과: 0.3분)
[detail] 100 / 500 편 처리 완료 (경과: 0.3분)
[detail] 150 / 500 편 처리 완료 (경과: 0.3분)
[detail] 200 / 500 편 처리 완료 (경과: 0.3분)
[detail] 250 / 500 편 처리 완료 (경과: 0.3분)
[detail] 300 / 500 편 처리 완료 (경과: 0.3분)
[detail] 350 / 500 편 처리 완료 (경과: 0.3분)
[detail] 400 / 500 편 처리 완료 (경과: 0.3분)
[detail] 450 / 500 편 처리 완료 (경과: 0.3분)
[detail] 500 / 500 편 처리 완료 (경과: 0.3분)
최종 샘플 데이터 행/열: (500, 26)

🎉 수집 완료!
📊 최종 데이터: 500편
💾 저장 완료: tmdb_movies_post_covid_may2023_500.csv

[데이터 요약]
        id                                   title  release_year  revenue  \
0  1140738                        The Bastard Sons          2024        0   
1  1114114                                   IB 71          2023        0   
2  1214469                             

In [3]:
df = pd.read_csv('tmdb_movies_post_covid_may2023_500.csv')
df

,id,imdb_id,title,original_title,original_language,spoken_languages,overview,tagline,release_date,runtime,...,genres,genre_ids,production_companies,production_countries,keywords,top_cast,directors,writers,providers,release_year
0,1140738,tt15665448,The Bastard Sons,The Bastard Sons,en,['English'],Life for a crew of organized criminals is disr...,NaN,2024-01-05,83,...,"['Crime', 'Thriller', 'Drama']","[80, 53, 18]","['A Different Path Films', 'Joe Minella Studio...",['United States of America'],[],"['Al Sapienza', 'Charles Malik Whitfield', 'An...",['Kevin Interdonato'],"['Kevin Interdonato', 'Glenn Rodriguez']","{'CA': ['Amazon Prime Video', 'Amazon Prime Vi...",2024
1,1114114,tt15073166,IB 71,IB 71,hi,['हिन्दी'],A two- front war between Indian intelligence a...,A top secret mission that made us win the 1971...,2023-05-12,117,...,"['Action', 'Thriller', 'War']","[28, 53, 10752]","['Reliance Entertainment', 'T-Series', 'Action...",['India'],[],"['Vidyut Jammwal', 'Dalip Tahil', 'Anupam Kher']",['Sankalp Reddy'],['Sankalp Reddy'],{'EG': ['Disney Plus']},2023
2,1214469,tt30321095,Ghostlight,Ghostlight,en,['English'],When a construction worker unexpectedly joins ...,NaN,2024-06-14,115,...,"['Comedy', 'Drama']","[35, 18]","['Little Engine', 'Runaway Train']",['United States of America'],"['therapy', 'construction worker', 'father dau...","['Keith Kupferer', 'Katherine Mallen Kupferer'...","['Alex Thompson', ""Kelly O'Sullivan""]","[""Kelly O'Sullivan""]","{'AE': ['TOD'], 'BA': ['HBO Max'], 'BG': ['HBO...",2024
3,1501204,NaN,I Am a Plaything,나는 노리개다,ko,['한국어/조선말'],"Hyo-min, who became a prostitute by choice, un...",NaN,2025-06-14,78,...,"['Drama', 'Romance']","[18, 10749]",[],[],['softcore'],"['Seung Ha', 'Woo Yeol', 'Do Hee']",['Lee Jung-hwan'],[],{'KR': ['TVING']},2025
4,894998,tt30564758,Albatross,Albatross,it,['Italiano'],Two friends at opposite ends of the political ...,NaN,2025-07-03,90,...,['Drama'],[18],"['One More Pictures', 'RAI Cinema', 'Friuli Ve...",['Italy'],"['biography', 'fashion']","['Francesco Centorame', 'Giancarlo Giannini', ...",['Giulio Base'],"['Giulio Base', 'Giulio Base', 'Giulio Base', ...",{},2025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,1431530,NaN,Woh Kaun Thi,Woh Kaun Thi,hi,['हिन्दी'],One Night when Anjali and Rohan were returning...,Death Is Just The Begining...,2025-02-05,118,...,['Horror'],[27],[],['India'],[],"['Punit Tejwani', 'Madhu Sharma', 'Smita Jaykar']",['Deepak Ramsay'],[],{},2025
496,1116144,tt33355095,High Stakes - A Night In The Ward,La scommessa - Una notte in corsia,it,['Italiano'],"It’s Ferragosto, Italy’s mid-August holiday, i...",NaN,2024-09-12,84,...,['Comedy'],[35],"['Italian International Film', 'I Wonder Pictu...",['Italy'],[],"['Carlo Buccirosso', 'Lino Musella', 'Nando Pa...",['Giovanni Dota'],"['Giovanni Dota', 'Giulia Magda Martinez', 'Ma...","{'CH': ['blue TV'], 'IT': ['Timvision'], 'RU':...",2024
497,1185776,tt23870264,Glisten and the Merry Mission,Glisten and the Merry Mission,en,['English'],"Follows Marzipan, a young elfling, and her mot...",It's about believing!,2023-11-03,78,...,"['Adventure', 'Animation', 'Family', 'Fantasy']","[12, 16, 10751, 14]","['Samuel Goldwyn Films', 'Build-a-Bear Enterta...",['United States of America'],"['santa claus', 'grizzly bear', 'reindeer', 'n...","['Leona Lewis', 'Chevy Chase', 'Julia Michaels']",['Cory Morrison'],"['Sharon Price John', 'Temple Mathews']","{'AE': ['TOD'], 'BH': ['TOD'], 'CA': ['Amazon ...",2023
498,1212411,tt24666718,Morningside,Morningside,en,[],"A powerful story about ambition, resilience, a...",NaN,2025-02-21,91,...,['Drama'],[18],['Ron & Aussie'],['Canada'],[],"['Kiana Madeira', 'Alex Mallari Jr.', 'Oluniké...",['Ron Dias'],"['Ron Dias', 'Joanne Jansen']","{'CA': ['Hollywood Suite', 'Hollywood Suite Am...",2025


In [9]:
df[['providers','budget','revenue']]

,providers,budget,revenue
0,"{'CA': ['Amazon Prime Video', 'Amazon Prime Vi...",0,0
1,{'EG': ['Disney Plus']},0,0
2,"{'AE': ['TOD'], 'BA': ['HBO Max'], 'BG': ['HBO...",0,672640
3,{'KR': ['TVING']},0,0
4,{},0,0
...,...,...,...
495,{},0,0
496,"{'CH': ['blue TV'], 'IT': ['Timvision'], 'RU':...",0,0
497,"{'AE': ['TOD'], 'BH': ['TOD'], 'CA': ['Amazon ...",0,0
498,"{'CA': ['Hollywood Suite', 'Hollywood Suite Am...",0,0


In [13]:
# providers 중 revenue 존재 여부 
df_2 = df[df['providers'] != "{}"]
print(f"providers 중 revenue 없는 경우: {df_2[df_2[('revenue')] == 0]['id'].count()}")
print(f"providers 중 revenue 있는 경우: {df_2[df_2[('revenue')] != 0]['id'].count()}")

providers 중 revenue 없는 경우: 227
providers 중 revenue 있는 경우: 54
